# Import Libraries

In [5]:
import torch # standard basic library for pytorch
import torch.autograd # need Variable class
import torch.nn as nn # need the basic neural net module
import torch.nn.functional as F # imported for the conv1d function

import torch.nn.parameter as Parameter

# Normal Convolution

In [6]:
convFilter = torch.FloatTensor([8,2]) # use [8,2] for test kernel
convFilter = torch.unsqueeze(convFilter,0) # unsqueeze twice to make filter a 3 dimensional 1x1x2 filter
convFilter = torch.unsqueeze(convFilter,0)
convInput = torch.unsqueeze(torch.autograd.Variable(torch.FloatTensor([3,7,6,4])),0) # use [3,7,6,4] as test input
convInput = torch.unsqueeze(convInput,0) #unsqueeze twice to make input 3D 1x1x4 for the Conv1D class
print(convInput)


conv = torch.nn.Conv1d(1,1,2,padding = 0, bias = False) # set bias to false for now since no learning is required yet
conv.weight = torch.nn.Parameter(convFilter) # set weight of conv filter to be what was specified
print(conv.weight)

convOutput = conv(convInput) # convolve input with filter --> this should give non-causal convolution
print(convOutput)

Variable containing:
(0 ,.,.) = 
  3  7  6  4
[torch.FloatTensor of size 1x1x4]

Parameter containing:
(0 ,.,.) = 
  8  2
[torch.FloatTensor of size 1x1x2]

Variable containing:
(0 ,.,.) = 
  38  68  56
[torch.FloatTensor of size 1x1x3]



# Causal Convolution

In [13]:
class CausalConv(torch.nn.Conv1d): # a class for causal convolution exclusively
    def __init__(self,in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super(CausalConv,self).__init__(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups, bias = bias) # initialise with Conv1d init variables
        self.causality_padding = kernel_size - 1 # padding size depends on kernel size; output at t depend on input t-n...input t
        
    def forward(self, inputs):
        inputs = F.pad(inputs, (self.causality_padding,0,0,0))
        return F.conv1d(inputs, self.weight)

# Test for Causal Convolution

In [14]:
causalConvInput = torch.ones(1,2,5)
print(causalConvInput)
# causalConvFilter = torch.ones(2,1,3)
# print(causalConvFilter)
causal_conv = CausalConv(2,2,3)
# causal_conv.weight = torch.nn.Parameter(causalConvFilter)
causal_conv_output = causal_conv(causalConvInput)

print(causal_conv_output)


(0 ,.,.) = 
  1  1  1  1  1
  1  1  1  1  1
[torch.FloatTensor of size 1x2x5]

Variable containing:
(0 ,.,.) = 
  0.0160 -0.4389 -0.2531 -0.2531 -0.2531
 -0.0249 -0.5494 -0.8015 -0.8015 -0.8015
[torch.FloatTensor of size 1x2x5]



# 1x1 Convolution

In [17]:
class OneConv(torch.nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size = 1, stride = 1, padding = 0, dilation = 1, groups = 1, bias = True):
        super(OneConv, self).__init__(in_channels, out_channels, kernel_size = kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups, bias = bias)
        
    def forward(self, inputs):
        return super(OneConv,self).forward(inputs)

# 1x1 Convolution Testing

In [33]:
oneInput = torch.autograd.Variable(torch.ones(2,1,5))
print(oneInput)
oneConv = OneConv(1,1)
oneOutput = oneConv(oneInput)

Variable containing:
(0 ,.,.) = 
  1  1  1  1  1

(1 ,.,.) = 
  1  1  1  1  1
[torch.FloatTensor of size 2x1x5]



# Wavenet

In [10]:
class WaveNet(nn.Module):
    def __init__(self):
        super(WaveNet, self).__init__()
        
    def forward(self, inputs):
        return inputs